In [6]:
import pickle 
import pandas as pd 

In [7]:
with open('../data/data_v2.pickle','rb') as f:
    buildings = pickle.load(f)

In [3]:
print(buildings[0:5])

[defaultdict(None, {'address': '993+union+avenue+bronx', 'amenities': [], 'type': 'C1', 'details': ['21 units', '5 stories', 'Built in 1905'], 'BIN': 2004845.0}), defaultdict(None, {'address': '90-26+171+street+queens', 'amenities': [], 'type': 'D1', 'details': ['68 units', '7 stories', '4 buildings', 'Built in 2013'], 'BIN': 4209764.0}), defaultdict(None, {'address': '1017+hegeman+avenue+brooklyn', 'amenities': ['NYC Storm Zone 3'], 'type': 'S4', 'details': ['4 units', '3 stories', 'Built in 1930', 'Community District 305', 'City Council District 42', 'Police Precinct 75'], 'BIN': 3098418.0}), defaultdict(None, {'address': '1794+anthony+avenue+bronx', 'amenities': [], 'type': 'D1', 'details': ['37 units', '5 stories', 'Built in 2002'], 'BIN': 2115634.0}), defaultdict(None, {'address': '279+lee+avenue+brooklyn', 'amenities': ['Live-in Super', 'Virtual Doorman', 'NYC Storm Zone 3'], 'type': 'D1', 'details': ['10 units', '7 stories', 'Built in 2004', 'Community District 301', 'City Counc

In [4]:
building = buildings[4]


In [12]:
print(building.keys())

dict_keys(['address', 'amenities', 'type', 'details', 'apt_list', 'apts', 'BIN'])


In [6]:
apartments = pd.DataFrame()
for b in buildings: 
    if b and 'apts' in b.keys():
        apt_list = b['apts']
        amenities = b['amenities']
        i = 1
        for a in apt_list: 
            price_df = a['price_history'][0]
            price_df['address'] = b['address']
            price_df['BIN'] = b['BIN']
            price_df['rooms'] = a['details'][0]
            price_df['bedrooms'] = a['details'][1]
            price_df['bath'] = a['details'][2]
            price_df['apt_num'] = i
            price_df['amenities'] = str(amenities)
            apartments = apartments.append(price_df)
            i += 1
            
        

In [4]:
building['apts']

NameError: name 'building' is not defined

In [7]:
apartments.columns

Index([0, 1, 2, 'address', 'BIN', 'rooms', 'bedrooms', 'bath', 'apt_num',
       'amenities'],
      dtype='object')

In [8]:
apartments.rename(columns = {0:'date', 1:'descr', 2:'rent_price'}, inplace = True)

In [9]:
apartments.head()

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities
0,10/02/2020,Off market temporarily,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,10/01/2020,Listing entered contract,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
2,09/25/2020,Listed by StayBK LLC,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
0,04/23/2017,Listing rented,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,03/07/2017,Listing entered contract,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."


In [10]:
apartments['date'] = pd.to_datetime(apartments['date'])

<ipython-input-61-f427e67b00fe>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  apartments['date'].describe()


count                   14472
unique                   3282
top       2017-07-18 00:00:00
freq                       51
first     2006-05-12 00:00:00
last      2021-03-12 00:00:00
Name: date, dtype: object

In [11]:
apartments['year'] = apartments['date'].apply(lambda x: x.year)

In [14]:
apartments_year_subset = apartments[((apartments['year'] >= 2016.0)&(apartments['year'] <= 2017.0))| (apartments['year'] == 2019.0)]

In [13]:
apartments_year_subset = apartments[(apartments['year'] <= 2017.0)| (apartments['year'] == 2019.0)]

In [15]:
apartments_grouped = apartments_year_subset.groupby(['BIN','apt_num']).filter(lambda x: x['year'].min() < 2018 and x['year'].max() >= 2019)

In [16]:
apartments_grouped

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year
0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0
1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0
7,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0
8,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0
1,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2019.0
...,...,...,...,...,...,...,...,...,...,...,...
9,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
10,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
11,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
12,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0


In [17]:
len(apartments_grouped['BIN'].unique())

123

In [18]:
len((apartments_grouped['BIN']+apartments_grouped['apt_num']).unique())

262

In [8]:
housing_df = pd.read_csv('../data/test_set.csv')

In [38]:
housing_df.columns

Index(['RegistrationID', 'BuildingID', 'BoroID', 'Boro', 'HouseNumber',
       'LowHouseNumber', 'HighHouseNumber', 'StreetName', 'StreetCode', 'Zip',
       'Block', 'Lot', 'BIN', 'CommunityBoard', 'LastRegistrationDate',
       'RegistrationEndDate', 'ADDRESS_ID', 'H_NO', 'the_geom', 'HNO_SUFFIX',
       'HYPHEN_TYP', 'SIDE_OF_ST', 'SPECIAL_CO', 'BOROCODE', 'ZIPCODE',
       'CREATED', 'MODIFIED', 'ST_NAME', 'HN_RNG_SUF', 'HN_RNG', 'PHYSICALID',
       'PRE_MODIFI', 'PRE_DIRECT', 'PRE_TYPE', 'POST_TYPE', 'POST_DIREC',
       'POST_MODIF', 'FULL_STREE', 'geometry', '250m_to_2018_dev',
       'no_proximity_newdev', 'lat', 'lon'],
      dtype='object')

In [241]:
apartments_grouped['BIN'] = apartments_grouped['BIN'].apply(int)

In [228]:
apartments_grouped['BIN'] = apartments_grouped['BIN'].apply(lambda x: x)

In [255]:
housing_df["BIN"] = housing_df["BIN"].apply(int)

In [234]:
housing_df.rename(columns = {"BuildingID":"BIN"}, inplace = True)

In [20]:
apart_join = apartments_grouped.merge(housing_df[['BIN', 'no_proximity_newdev']], on = ['BIN'])

In [21]:
apart_join

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year,no_proximity_newdev
0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True
1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True
2,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True
3,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True
4,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2019.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False
1570,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False
1571,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False
1572,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0,False


In [36]:
apart_join.groupby(['BIN','apt_num'])["no_proximity_newdev"]

In [37]:
apart_join.to_csv("../data/apartment_data_compiled.csv")
